In [4]:
import pygame
import math

# Initialize pygame
pygame.init()

# Define the window dimensions
WIDTH, HEIGHT = 800, 800

# Create the game window
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Space Exploration")

# Define colors
WHITE = (255, 255, 255)
YELLOW = (255, 255, 0)
BLUE = (100, 149, 237)
RED = (188, 39, 50)
DARK_GREY = (80, 78, 81)

# Define a font for text rendering
FONT = pygame.font.SysFont("comicsans", 16)

class Planet:
    AU = 149.6e6 * 1000
    G = 6.67428e-11
    SCALE = 250 / AU  # 1AU = 100 pixels
    TIMESTEP = 3600 * 24  # 1 day

    def __init__(self, x, y, radius, color, mass):
        # Initialize planet properties
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.mass = mass

        # Orbit path
        self.orbit = []

        # Sun indicator
        self.sun = False

        # Distance to the sun
        self.distance_to_sun = 0

        # Velocity components
        self.x_vel = 0
        self.y_vel = 0

    def draw(self, win):
        # Calculate screen coordinates
        x = self.x * self.SCALE + WIDTH / 2
        y = self.y * self.SCALE + HEIGHT / 2

        # Draw orbit path
        if len(self.orbit) > 2:
            updated_points = [(px * self.SCALE + WIDTH / 2, py * self.SCALE + HEIGHT / 2) for px, py in self.orbit]
            pygame.draw.lines(win, self.color, False, updated_points, 2)

        # Draw the planet
        pygame.draw.circle(win, self.color, (x, y), self.radius)

        # Display distance text
        if not self.sun:
            distance_text = FONT.render(f"{round(self.distance_to_sun / 1000, 1)} km", 1, WHITE)
            win.blit(distance_text, (x - distance_text.get_width() / 2, y - distance_text.get_height() / 2))

    def attraction(self, other):
        # Calculate gravitational attraction between two planets
        other_x, other_y = other.x, other.y
        distance_x = other_x - self.x
        distance_y = other_y - self.y
        distance = math.sqrt(distance_x ** 2 + distance_y ** 2)

        if other.sun:
            self.distance_to_sun = distance

        force = self.G * self.mass * other.mass / distance**2
        theta = math.atan2(distance_y, distance_x)
        force_x = math.cos(theta) * force
        force_y = math.sin(theta) * force
        return force_x, force_y

    def update_position(self, planets):
        # Update planet position based on gravitational forces
        total_fx = total_fy = 0
        for planet in planets:
            if self == planet:
                continue

            fx, fy = self.attraction(planet)
            total_fx += fx
            total_fy += fy

        self.x_vel += total_fx / self.mass * self.TIMESTEP
        self.y_vel += total_fy / self.mass * self.TIMESTEP

        self.x += self.x_vel * self.TIMESTEP
        self.y += self.y_vel * self.TIMESTEP
        self.orbit.append((self.x, self.y))

class Moon:
    def __init__(self, planet, distance, radius, color, orbital_period):
        self.planet = planet
        self.distance = distance
        self.radius = radius
        self.color = color
        self.orbital_period = orbital_period
        self.angle = 0

    def update_position(self):
        self.angle += (2 * math.pi) / self.orbital_period
        self.planet_x = self.planet.x + self.distance * math.cos(self.angle)
        self.planet_y = self.planet.y + self.distance * math.sin(self.angle)

    def draw(self, win):
        self.update_position()
        x = self.planet_x * Planet.SCALE + WIDTH / 2
        y = self.planet_y * Planet.SCALE + HEIGHT / 2
        pygame.draw.circle(win, self.color, (int(x), int(y)), self.radius)

def main():
    run = True
    clock = pygame.time.Clock()

    # Create planets
    sun = Planet(0, 0, 30, YELLOW, 1.98892 * 10**30)
    sun.sun = True

    earth = Planet(-1 * Planet.AU, 0, 16, BLUE, 5.9742 * 10**24)
    earth.y_vel = 29.783 * 1000

    mars = Planet(-1.524 * Planet.AU, 0, 12, RED, 6.39 * 10**23)
    mars.y_vel = 24.077 * 1000

    mercury = Planet(0.387 * Planet.AU, 0, 8, DARK_GREY, 3.30 * 10**23)
    mercury.y_vel = -47.4 * 1000

    venus = Planet(0.723 * Planet.AU, 0, 14, WHITE, 4.8685 * 10**24)
    venus.y_vel = -35.02 * 1000

    planets = [sun, earth, mars, mercury, venus]

    # Create moons for planets
    moon_earth = Moon(earth, 0.1 * Planet.AU, 5, WHITE, 27.3)  # Example moon for Earth
    moons = [moon_earth]  # Add moons to this list

    while run:
        clock.tick(60)
        WIN.fill((0, 0, 0))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

        for planet in planets:
            planet.update_position(planets)
            planet.draw(WIN)

        for moon in moons:
            moon.draw(WIN)

        pygame.display.update()

    pygame.quit()

main()
